# Simple

In [4]:
import random
import json

def synth_dict_simple(num_examples=1100, num_dicts=150, train_split=1000):
    dataset = []
    for _ in range(num_examples):
        # Генерация золотого ключа
        golden_key = random.randint(100, 9999)
        
        # Случайное значение для золотого ключа
        golden_value = random.randint(100, 9999)
        
        # Выбор словаря для золотого ключа
        golden_dict_id = random.randint(1, num_dicts)
        
        # Генерация всех словарей
        context = []
        for dict_id in range(1, num_dicts + 1):
            size = random.randint(2, 4)
            
            entries = {}
            if dict_id == golden_dict_id:
                entries[golden_key] = golden_value
                size = size - 1
                
            # Генерация остальных ключей
            existing = set()
            for _ in range(size):
                for _ in range(10):  # Попытки генерации
                    key = random.randint(100, 9999)
                    
                    # Проверка на конфликты
                    if key == golden_key or key in existing:
                        continue
                    
                    existing.add(key)
                    entries[key] = random.randint(100, 9999)
                    break
            
            # Форматирование словаря
            dict_str = f"Cловарь [{dict_id}] {{"
            dict_str += ", ".join(f"{k}: {v}" for k, v in entries.items())
            dict_str += "}"
            context.append(dict_str)
        
        # Формирование ответа
        answer = (
            f'{{\n'
            f'  "значение": {golden_value},\n'
            f'  "номер_словаря": {golden_dict_id}\n'
            f'}}'
        )
                
        dataset.append({
            "key": golden_key,
            "context": "\n".join(context),
            "answer": answer
        })
    
    return dataset[:train_split], dataset[train_split:]

In [5]:
train_data, val_data = synth_dict_simple()

In [6]:
print(train_data[0]["answer"])

{
  "значение": 4765,
  "номер_словаря": 24
}


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct") #meta-llama/Llama-3.2-1B-Instruct

In [5]:
print(len(tokenizer.encode(train_data[0]['context'], add_special_tokens=False)))

6952


In [ ]:
print(len(tokenizer.encode(train_data[0]['context'], add_special_tokens=False))) # llama

4842


# Hard

In [8]:
def synth_dict_hard(num_examples=1600, num_dicts=80, train_split=1500):
    dataset = []
    for _ in range(num_examples):
        # Генерация золотых чисел
        num_golden = random.choice([3, 4])
        golden_numbers = random.sample(range(100, 1000), num_golden)
        
        # Случайное значение для золотого ключа
        golden_value = random.randint(1000, 9999)
        
        # Выбор словаря для золотого ключа
        golden_dict_id = random.randint(1, num_dicts)
        
        # Генерация всех словарей
        context = []
        for dict_id in range(1, num_dicts + 1):
            size = random.randint(2, 4)
            
            entries = {}
            if dict_id == golden_dict_id:
                # Генерация золотого ключа
                shuffled = list(golden_numbers)
                random.shuffle(shuffled)
                golden_key = tuple(shuffled)
                entries[golden_key] = golden_value
                size = size - 1
                
            # Генерация остальных ключей
            existing = set()
            for _ in range(size):
                for _ in range(10):  # Попытки генерации
                    length = random.choice([3, 4])
                    nums = tuple(random.choices(range(100, 1000), k=length))
                    sorted_nums = tuple(sorted(nums))
                    
                    # Проверка на конфликты
                    if length == num_golden and sorted_nums == tuple(sorted(golden_numbers)):
                        continue
                    if sorted_nums in existing:
                        continue
                    
                    existing.add(sorted_nums)
                    entries[nums] = random.randint(1000, 9999)
                    break
            
            # Форматирование словаря
            dict_str = f"Cловарь [{dict_id}] {{"
            dict_str += ", ".join(f"{k}: {v}" for k, v in entries.items())
            dict_str += "}"
            context.append(dict_str)
        
        # Формирование ответа
        answer = (
            f'{{\n'
            f'  "ключ": {golden_key},\n'
            f'  "значение": {golden_value},\n'
            f'  "номер_словаря": {golden_dict_id}\n'
            f'}}'
        )
                
        dataset.append({
            "key": golden_numbers,
            "context": "\n".join(context),
            "answer": answer
        })
    
    return dataset[:train_split], dataset[train_split:]

In [9]:
train_data, val_data = synth_dict_hard()

In [10]:
print(train_data[0]["answer"])

{
  "ключ": (752, 574, 382),
  "значение": 6444,
  "номер_словаря": 41
}


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

In [10]:
print(len(tokenizer.encode(train_data[0]['context'], add_special_tokens=False)))

6561


In [ ]:
print(len(tokenizer.encode(train_data[0]['context'], add_special_tokens=False))) # llama

4326


# Save

In [11]:
SYSTEM_PROMPT = (
    "Ты — экспертная система Compressa RAG, предоставляющая точные и релевантные ответы на вопросы"
)

def get_simple_prompt(key, context):
    prompt = (
        f'Выполни задание, используя список словарей ниже.\n\n{context}\n\n'
        'Выше приведен список словарей, в которых каждый ключ и значение являются целыми числами.\n'
        f'Скажи значение по ключу {key}, а также номер словаря, в котором он находится.\n'
        'Ответ должен быть написан в формате json с полями "значение", "номер_словаря".'
    )
    return prompt

def get_hard_prompt(key, context):
    prompt = (
        f'Выполни задание, используя список словарей ниже.\n\n{context}\n\n'
        'Выше приведен список словарей, в которых каждый ключ представляет собой кортеж целых чисел, а каждое значение представляет собой целое число.\n'
        f'Скажи ключ, содержащий целые числа {key} (не обязательно по порядку), его значение и словарь, в котором он находится.\n'
        'Ответ должен быть написан в формате json c полями "ключ", "значение", "номер словаря".'
    )
    return prompt

In [12]:
train_data_simple, val_data_simple = synth_dict_simple()

with open('train/synth_needle_simple_train.jsonl', 'w') as f:
    for item in train_data_simple:
        data = ({
            "messages": [
                {"content": SYSTEM_PROMPT, "role": "system"},
                {"content": get_simple_prompt(item["key"], item["context"]), "role": "user"},
                {"content": item["answer"], "role": "assistant"}
            ],
        })
        f.write(json.dumps(data) + '\n')

with open('val/synth_needle_simple_val.jsonl', 'w') as f:
    for item in val_data_simple:
        data = ({
            "messages": [
                {"content": SYSTEM_PROMPT, "role": "system"},
                {"content": get_simple_prompt(item["key"], item["context"]), "role": "user"},
                {"content": item["answer"], "role": "assistant"}
            ],
        })
        f.write(json.dumps(data) + '\n')

In [13]:
train_data_hard, val_data_hard = synth_dict_hard()

with open('train/synth_needle_hard_train.jsonl', 'w') as f:
    for item in train_data_hard:
        data = ({
            "messages": [
                {"content": SYSTEM_PROMPT, "role": "system"},
                {"content": get_hard_prompt(item["key"], item["context"]), "role": "user"},
                {"content": item["answer"], "role": "assistant"}
            ],
        })
        f.write(json.dumps(data) + '\n')

with open('val/synth_needle_hard_val.jsonl', 'w') as f:
    for item in val_data_hard:
        data = ({
            "messages": [
                {"content": SYSTEM_PROMPT, "role": "system"},
                {"content": get_hard_prompt(item["key"], item["context"]), "role": "user"},
                {"content": item["answer"], "role": "assistant"}
            ],
        })
        f.write(json.dumps(data) + '\n')

In [14]:
with open("train/synth_needle_simple_train.jsonl", "r") as input:
    print(f"simple train: {len(input.readlines())}")
    
with open("val/synth_needle_simple_val.jsonl", "r") as input:
    print(f"simple val: {len(input.readlines())}")

with open("train/synth_needle_hard_train.jsonl", "r") as input:
    print(f"hard train: {len(input.readlines())}")

with open("val/synth_needle_hard_val.jsonl", "r") as input:
    print(f"hard val: {len(input.readlines())}")

simple train: 1000
simple val: 100
hard train: 1500
hard val: 100


# Test

In [6]:
import random
import json

with open("val/synth_needle_hard_val.jsonl", "r") as input:
    item = json.loads(input.readlines()[2])

print(item["messages"][1]["content"])

Выполни задание, используя список словарей ниже.

Cловарь [1] {(616, 496, 991, 901): 9214, (568, 258, 796): 6616, (366, 837, 616): 1064}
Cловарь [2] {(419, 178, 831, 715): 8888, (758, 193, 450): 5517}
Cловарь [3] {(554, 589, 604): 4464, (480, 193, 902): 5399}
Cловарь [4] {(155, 649, 403): 2225, (739, 665, 606): 3125}
Cловарь [5] {(545, 716, 414, 486): 1288, (510, 890, 573): 3753}
Cловарь [6] {(257, 239, 459, 333): 2855, (667, 228, 354): 2855, (296, 201, 486, 168): 4454, (724, 784, 520, 850): 6850}
Cловарь [7] {(548, 253, 361, 174): 4953, (893, 227, 913, 348): 9076, (439, 659, 560, 185): 2946}
Cловарь [8] {(837, 447, 875, 553): 5979, (876, 556, 886): 8120}
Cловарь [9] {(810, 977, 727, 169): 8841, (284, 930, 819): 9822, (900, 412, 255, 752): 3374, (134, 730, 820, 931): 2574}
Cловарь [10] {(186, 201, 933, 480): 1868, (218, 110, 390, 784): 3728}
Cловарь [11] {(234, 942, 205): 4438, (443, 659, 483, 155): 7772, (763, 355, 378): 7316}
Cловарь [12] {(893, 198, 191, 601): 3975, (255, 852, 945):

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct", padding_side='left')

BOS_TOKEN = tokenizer.bos_token if tokenizer.bos_token else tokenizer.additional_special_tokens[0] 
print(f"BOS_TOKEN={BOS_TOKEN}")

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",  # Путь к объединенной модели
    device_map="auto"
)
model.eval()


messages = item["messages"]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

BOS_TOKEN=<|im_start|>


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
print(response)

Вот ответ в требуемом формате JSON:

```json
{
  "ключ": (634, 949, 135, 225),
  "значение": 6808,
  "номер_словаря": 30
}
```


In [11]:
print(item["messages"][2]["content"])

{
  "ключ": (634, 949, 135, 225),
  "значение": 6808,
  "номер_словаря": 30
}
